In [ ]:
!pip install python-dotenv


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads variables from .env


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sessions = pd.read_csv('website_sessions.csv')
orders = pd.read_csv('orders.csv')
pageviews = pd.read_csv('website_pageviews.csv')
order_items = pd.read_csv('order_items.csv')
order_item_refunds = pd.read_csv('order_item_refunds.csv')
products = pd.read_csv('products.csv')


# 1. Data Cleaning
## 1.1 Check structure and datatypes


In [ ]:
sessions.info()
orders.info()
pageviews.info()
order_items.info()
order_item_refunds.info()
products.info()

we can see that the created_at column is suppose to be in datetime datatype but its in object. So, we have to convert the datatype into date format.
If it is not converted into datetime then monthly trend maynot work and conversion over time will fail.

### 1.1.1 Convert created_at to datetime format


In [ ]:
sessions['created_at'] = pd.to_datetime(sessions['created_at'])
orders['created_at'] = pd.to_datetime(orders['created_at'])
pageviews['created_at'] = pd.to_datetime(pageviews['created_at'])
order_items['created_at'] = pd.to_datetime(order_items['created_at'])
order_item_refunds['created_at'] = pd.to_datetime(order_item_refunds['created_at'])


In [ ]:
sessions.info()
orders.info()
pageviews.info()

In [ ]:
# checking min and max time span
print(sessions['created_at'].min(), sessions['created_at'].max())
print(orders['created_at'].min(), orders['created_at'].max())
print(pageviews['created_at'].min(), pageviews['created_at'].max())


We can see that the records are from 2012 april 19 to 2015 april 19. 

## 1.2. Categorical Data Cleaning(UTM and Channels)
Even when there are no nulls, UTM field may have NaN, empty strings or inconsistent values.

In [ ]:
# inspect UTM cloumns
sessions[['utm_source', 'utm_campaign', 'utm_content', 'device_type']].head(10)


In [ ]:
sessions['utm_source'].unique()



In [ ]:
sessions['utm_campaign'].unique()

In [ ]:
sessions['utm_content'].unique()

In [ ]:
sessions['device_type'].unique()

In [ ]:
sessions[['utm_source','utm_campaign','utm_content']].isnull().sum()


In [ ]:
sessions['utm_source'] = sessions['utm_source'].fillna('direct')
sessions['utm_campaign'] = sessions['utm_campaign'].fillna('none')
sessions['utm_content'] = sessions['utm_content'].fillna('none')


In [ ]:
sessions[['utm_source','utm_campaign','utm_content']].isnull().sum()

In [ ]:
# create marketing channel column
def channel_group(row):
    if row['utm_source'] == 'direct' :
        return 'direct'
    elif row ['utm_source'] in ['gsearch', 'bsearch'] :
        return 'paid_search'
    elif row ['utm_source'] == 'socialbook' :
        return 'paid_social'
    else:
        return 'other'

sessions['channel_group'] = sessions.apply(channel_group, axis=1)





#Or 

mapping = {
    'direct': 'direct',
    'gsearch': 'paid_search',
    'bsearch': 'paid_search',
    'socialbook': 'paid_social'
}

sessions['channel_group'] = sessions['utm_source'].map(mapping).fillna('other')

In [ ]:
sessions['channel_group'].value_counts()


This result shows how website sessions are distributed across major marketing channels after standardizing UTM data. Paid Search is the dominant channel with 378,858 sessions, indicating that the majority of traffic is driven by search advertising on platforms such as Google and Bing. This suggests a strong reliance on paid search for acquisition, making its performance critical to overall traffic and revenue. Direct traffic accounts for 83,328 sessions, reflecting brand strength and repeat usage through untagged visits such as bookmarks or direct URL access, and typically represents a lower-cost, higher-intent audience. Paid Social contributes 10,685 sessions, making it a smaller but strategically important channel, often associated with awareness or experimental campaigns.

## 1.3 Validate primary keys and table relationships
If IDs are duplicated or relationships are broken, conversion rates, revenue, and funnel metrics will be wrong — even if code runs without errors.

### 1.3.1 Validate Primary keys(uniqueness check)

In [ ]:
sessions['website_session_id'].is_unique

In [ ]:
orders['order_id'].is_unique

In [ ]:
pageviews['website_pageview_id'].is_unique


In [ ]:
order_items['order_item_id'].is_unique


In [ ]:
order_item_refunds['order_item_refund_id'].is_unique


### 1.3.2 Validate Table relationships
Confirm tables are link correctly or not.

In [ ]:
orders['website_session_id'].isin(sessions['website_session_id']).all()


In [ ]:
pageviews['website_session_id'].isin(sessions['website_session_id']).all()


In [ ]:
order_items['order_id'].isin(orders['order_id']).all()


In [ ]:
order_item_refunds['order_item_id'].isin(order_items['order_item_id']).all()


### 1.3.3 Cardinality Check


In [ ]:
# how many orders per session?

orders.groupby('website_session_id')['order_id'].count().describe()


In [ ]:
# how many pageviews pers sessions?
pageviews.groupby('website_session_id')['website_pageview_id'].count().describe()

Primary keys and table relationships are vailidated to ensure that each dataset had unique identifiers and that foreign keys correctly linked sessions, orders, pageviews, and order items. This step ensures that joins are accurate and that conversion and revenue metrics are not inflated due to duplicated or orphaned records.

# 2. Exploratory Data Analysis (EDA)
In this section, we will understand trend and patterns before drawing conclusions. We will analyze the following:
1. Sessions trend
2. Orders trend
3. Conversion rate trend
4. Channel-level performance

In [ ]:
# create time features(monthly)
sessions['month'] = sessions['created_at'].dt.to_period('M')
orders['month'] = orders['created_at'].dt.to_period('M')


## 2.1 Website sessions trend


In [ ]:
session_trend = (
    sessions
    .groupby('month')['website_session_id']
    .nunique()
)

session_trend


In [ ]:
session_trend.plot(
    title = 'Monthly Website Sessions',
    figsize = (10,5)
)

## 2.2 Orders trend

In [ ]:
orders_trend = (
    orders
    .groupby('month')['order_id']
    .nunique()
)

orders_trend

In [ ]:
orders_trend.plot(
    title = 'Monthly orders',
    figsize = (10,5)
)
    

## 2.3 Session to order Conversion Rate

In [ ]:
#merge session and order(left join)
session_orders = sessions.merge(
    orders[['order_id', 'website_session_id']],
    on = 'website_session_id',
    how = 'left'
)

session_orders.head(5)

In [ ]:
# calculate conversion rate
conversion_monthly = (
    session_orders
    .groupby('month')
    .agg(
        sessions = ('website_session_id', 'nunique'),
        orders = ('order_id', 'nunique')
    )
)

conversion_monthly['conversion_rate'] = (
    conversion_monthly['orders'] / conversion_monthly['sessions']
)

conversion_monthly

In [ ]:
conversion_monthly['conversion_rate'].plot(
    title = 'Monthly Session-to-Order Conversion Rate',
    figsize = (10,5)
)

## 2.4 Channel- level traffic volume

In [ ]:
channel_sessions = (
    sessions
    .groupby('channel_group')['website_session_id']
    .nunique()
    .sort_values(ascending=False)
)

channel_sessions

Website sessions have grown steadily over time, driven by increased marketing activity, with a noticeable spike toward the end of 2014. Orders have followed a similar upward trend, indicating that traffic growth is translating into real sales rather than low-quality visits. Importantly, the session-to-order conversion rate has improved significantly over the same period, rising from roughly 3% to over 7%, which suggests improvements in website performance, traffic quality, or both. Overall, this indicates that business growth is being driven by both higher traffic volume and stronger conversion efficiency.

## 2.5 Channel Level Performance Analysis
Which channel converts best? Which generates the most revenue? Which is most efficient?

## 2.5.1 Sessions, Orders and Conversion Rate by Channel


In [ ]:
# merge session and orders
channel_perf = (
    sessions.merge(
        orders[['order_id', 'website_session_id', 'price_usd']],
        on = 'website_session_id',
        how = 'left'
    )
)

In [ ]:
# aggregate by channel
channel_summary = (
    channel_perf
    .groupby('channel_group')
    .agg(
        sessions=('website_session_id', 'nunique'),
        orders=('order_id', 'nunique'),
        revenue=('price_usd', 'sum')
    )
)

channel_summary['conversion_rate'] = (
    channel_summary['orders'] / channel_summary['sessions']
)

channel_summary

    

In [ ]:
# Revenue per session
channel_summary['revenue_per_session'] = (
    channel_summary['revenue'] / channel_summary['sessions']
)

channel_summary


Paid search is the main driver of traffic and revenue, making it the company’s primary growth channel, while direct traffic is the most efficient, showing the highest conversion rate and revenue per session due to strong brand-driven demand. Paid social contributes relatively little traffic and revenue and has the lowest efficiency, indicating a need for optimization or reduced investment.

In [ ]:
pip install psycopg2-binary sqlalchemy


In [ ]:
sessions_sql = sessions.drop(columns=['month'], errors='ignore')
orders_sql = orders.drop(columns=['month'], errors='ignore')


In [ ]:
from sqlalchemy import create_engine

# Step 1: Database connection details
username = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host = os.getenv("PG_HOST")
port = os.getenv("PG_PORT")
database = os.getenv("PG_DATABASE")

# Create PostgreSQL engine
engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)

# Step 2: Load DataFrame into PostgreSQL
tables = {
    "website_sessions": sessions_sql,
    "website_pageviews": pageviews,
    "orders": orders_sql,
    "order_items": order_items,
    "order_item_refunds": order_item_refunds,
    "products": products
}

for table_name, df in tables.items():
    df.to_sql(
        table_name,
        engine,
        if_exists="replace",
        index=False
    )
    
    print(f"Loaded table: {table_name}")


